In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# 403 forbidden rbac role&rolebinding

from kubernetes import client, config as kube_config
import yaml
import json
from conf import config


class KubernetesAPI(object):
    def __init__(self, name=None, namespace=None):
        try:
            kube_config.load_kube_config()
        except:
            # kube_config.load_incluster_config()
            self.configuration = client.Configuration()
            self.configuration.host = config.KUBE_HOST
            self.configuration.verify_ssl = False
            with open('/var/run/secrets/kubernetes.io/serviceaccount/token') as f:
                self.configuration.api_key['authorization'] = 'Bearer %s' % f.read()
        self.name = name
        self.namespace = namespace

    def create_resource(self):
        raise NotImplementedError

    def generate_body(self, config_file, args):
        body_tpl = yaml.load(config_file)
        body_str = json.dumps(body_tpl)
        body_str = body_str.replace(
            '"namespace": null', '"namespace": %s' % (self.namespace)
        )
        body_str = body_str.replace(
            '"name": null', '"name": %s' % (self.name)
        )
        body_str = body_str.replace(
            '"args": null', '"args": %s' % json.dumps(args)
        )
        body = yaml.safe_load(body_str)
        return body


class KubernetesJobAPI(KubernetesAPI):

    def __init__(self, name=None, namespace=None):
        super(KubernetesJobAPI, self).__init__(name, namespace)
        try:
            self.api_instance = client.BatchV1Api(client.ApiClient(self.configuration))
        except:
            self.api_instance = client.BatchV1Api()

    def create_resource(self, config_file, args):
        with open(config_file) as f:
            body = self.generate_body(f, args)
            self.api_instance.create_namespaced_job(
                body=body,
                namespace=self.namespace,
            )

    def get_resource(self, job_name):
        resource = self.api_instance.read_namespaced_job(
            name=job_name,
            namespace=self.namespace,
        )
        return resource


class KubernetesCronJobAPI(KubernetesAPI):

    def __init__(self, name=None, namespace=None):
        super(KubernetesCronJobAPI, self).__init__(name, namespace)
        self.api_instance = client.BatchV1beta1Api()

    def create_resource(self, config_file, args):
        with open(config_file) as f:
            body = self.generate_body(f, args)
            self.api_instance.create_namespaced_cron_job(
                body=body,
                namespace=self.namespace,
            )

    def get_resource(self, cron_job_name):
        resource = self.api_instance.read_namespaced_cron_job(
            name=cron_job_name,
            namespace=self.namespace,
        )
        return resource

if __name__ == '__main__':
    pass
